In [1]:
import math

%load_ext Cython

from Bio import PDB

repository = PDB.PDBList()
parser = PDB.PDBParser()

repository.retrieve_pdb_file('1TUP', pdir='.')
p53_1tup = parser.get_structure('P 53', 'pdb1tup.ent')

Structure exists: './1tup.cif' 


/home/saams4u/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6146.
  warnings.warn(
/home/saams4u/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 6147.
  warnings.warn(
/home/saams4u/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6148.
  warnings.warn(
/home/saams4u/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 6149.
  warnings.warn(
/home/saams4u/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 6171.
  warnings.warn(
/home/saams4u/.local/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 618

In [2]:
def get_distance(atoms):
    atoms = list(atoms)
    natoms = len(atoms)
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i].coord
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j].coord
            my_dist = math.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)

In [5]:
%time get_distance(p53_1tup.get_atoms())

CPU times: user 3min 29s, sys: 325 ms, total: 3min 30s
Wall time: 3min 49s


In [4]:
# %%cython
import math

def get_distance_cython_0(atoms):
    atoms = list(atoms)
    natoms = len(atoms)
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i].coord
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j].coord
            my_dist = math.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)
            
# %time get_distance_cython_0(p53_1tup.get_atoms())

In [5]:
%%cython
cimport cython

from libc.math cimport sqrt, pow

cdef double get_dist_cython(double xi, double yi, double zi, 
                            double xj, double yj, double zj):
    return sqrt(pow(xi - xj, 2) + pow(yi - yj, 2) + pow(zi - zj, 2))

def get_distance_cython_1(object atoms):
    natoms = len(atoms)
    cdef double x1, xj, yi, yj, zi, zj
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i]
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j]
            my_dist = get_dist_cython(xi, yi, zi, xj, yj, zj)
            
# %timeit get_distance_cython_1([atom.coord for atom in p53_1tup.get_atoms()])

In [6]:
from numba import float_
from numba.decorators import jit

get_distance_numba_0 = jit(get_distance)

%time get_distance_numba_0(p53_1tup.get_atoms())

<ipython-input-6-f6815e8a190b>:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit
<ipython-input-6-f6815e8a190b>:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit
<ipython-input-2-c5a25c8472f0>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_distance" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-2-c5a25c8472f0> (2)

File "<ipython-input-2-c5a25c8472f0>", line 2:
def

CPU times: user 3min 49s, sys: 586 ms, total: 3min 50s
Wall time: 4min 28s


In [7]:
@jit 
def get_dist_numba(xi, yi, zi, xj, yj, zj):
    return math.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)

def get_distance_numba_1(atoms):
    natoms = len(atoms)
    for i in range(natoms - 1):
        xi, yi, zi = atoms[i]
        for j in range(i + 1, natoms):
            xj, yj, zj = atoms[j]
            my_dist = get_dist_numba(xi, yi, zi, xj, yj, zj)
            
%timeit get_distance_numba_1([atom.coord for atom in p53_1tup.get_atoms()])

36.2 s ± 466 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
